Um das Model zu testen, ändern Sie den Dateinamen im Code-Teil unter "Unabhängige Validierung".

# Bibliotheken importieren

In [ ]:
# Data Science + Mathe Bibliotheken
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sb

# Metriken
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.metrics import max_error

# Validierung
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# Modelle
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor

# Warnungen 
import warnings

# MAPE definieren

In [ ]:
#https://www.statology.org/mape-python/

def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

# Anomalien entfernen und Daten bereinigen

In [ ]:
housing_data = pd.read_csv('DatenAusgegeben1.2_UTF8_manuell.csv', sep=";", keep_default_na=False)

warnings.filterwarnings('ignore')

# Bereinigung und Feature Engeneering

# für €/qm² für besseren Vergleich der Daten
housing_data["Preisproqm"] = housing_data["Preis"]/housing_data["Wohnflaeche in qm"]

# Monate seit 2136
housing_data["Monate"] = (housing_data["Verkaufsjahr"]-2136)*12 + housing_data["Verkaufsmonat"] -1

# Werte der Heizungsqualitaet in Zahlen umwandeln
housing_data["Heizungsqualitaet"] = [1 if x=="Schl" else 2 if x=="Ud" else 3 if x=="Ty" else 4 if x=="Gut" else 5 for x in housing_data["Heizungsqualitaet"]]

# Zustand, 1-10 wird auf 1-5 gemapt, das ist zwar ein bisschen ungenauer, aber einfacher zu vergleichen
housing_data["Zustandf"] = [int(x/2) + x%2 for x in housing_data["Zustand"]]

# Alle Zustände werden von 1...X auf 0....X-1 geschoben
housing_data["HeizungsqualitaetN"] = [ x-1 for x in housing_data["Heizungsqualitaet"]]
housing_data["ZustandN"] = [ x-1 for x in housing_data["Zustandf"]]
housing_data["FassadeN"] = [ x-1 for x in housing_data["Zustand Fassade"]]
housing_data["KuecheN"] = [ x-1 for x in housing_data["Kuechenqualitaet"]]

# Klimaanlage Ja/Nein  -> 1/0
housing_data["Klimaanlage"] = [1 if x == "Y" else 0 for x in housing_data["Klimaanlage"]]

# Besonders Große Daten beim Preis + Wohnfläche in qm rausschmeißen
max_value = housing_data["Preis"].unique()[-1:][0]
housing_data[housing_data["Preis"] != max_value]

max_value = housing_data["Wohnflaeche in qm"].unique()[-1:][0]
housing_data[housing_data["Wohnflaeche in qm"] != max_value]

max_value = housing_data["Grundstueck in qm"].unique()[-2:][0]
housing_data[housing_data["Grundstueck in qm"] != max_value]


# NA in Garage Typ wird zu "Keine Garage"
housing_data["Garage Typ"] = ['keine Garage' if x=='NA' else x for x in housing_data["Garage Typ"]]


print(housing_data.keys())

# Merkmale auswählen

In [ ]:
# X und Y aus dem Datenset auslesen

# Zu testende Merkmale festlegen (X)
features = [ "Grundstueck in qm", "Wohnflaeche in qm","ZustandN","KuecheN", "HeizungsqualitaetN", "FassadeN",
            "Klimaanlage", "Gebaut", "Garagenkapazitaet", "Raeume"]
# Preis (Y)
predict= "Preis"

X = {}
for feature in features:
    X[feature] = housing_data[feature]
X = pd.DataFrame(X)

Y = pd.DataFrame(housing_data[predict])

# Regression

In [ ]:
# K-fache Cross-Validierung
split = 10
Kfold = KFold(n_splits=split, random_state=89)
kfold = Kfold.split(X, Y)

model = {"data":[], "scores":{}, "models" : {}, "best" : {}, "types":["osl", "lasso", "ridge", "gbt", "rf", "sgd"]}
for mtype in model["types"]:
    model["scores"][mtype] = []
    model["models"][mtype] = []


for k, (train, test) in enumerate(kfold):
    model["models"]["osl"].append(linear_model.LinearRegression())
    model["models"]["lasso"].append(linear_model.Lasso())
    model["models"]["ridge"].append(linear_model.Ridge())
    model["models"]["gbt"].append(GradientBoostingRegressor(
        max_depth=4, n_estimators=100, learning_rate=0.15, loss="ls"))
    model["models"]["rf"].append(RandomForestRegressor(max_depth=10, n_estimators=10, criterion="mse"))
    model["models"]["sgd"].append(SGDRegressor())
    
    # Polynomielle Regression
    poly = PolynomialFeatures(degree=3)
    x_poly = poly.fit_transform(X.iloc[train, :])
    poly.fit(X.iloc[train, :], Y.iloc[train])
    x_poly_test = poly.fit_transform(X.iloc[test, :])
    y_train = Y.iloc[train]
    y_test = Y.iloc[test]
    model["data"].append({"xpoly": x_poly, "x_train": X.iloc[train, :], "y_train": y_train, 
                          "x_poly_test": x_poly_test, "x_test": X.iloc[test, :], "y_test": y_test})
    
    for mtype in model["types"]:
        curr_model = model["models"][mtype][k]
        curr_model.fit(x_poly, y_train)
        score = r2_score(curr_model.predict(x_poly_test), y_test)
        model["scores"][mtype].append(score)
        
print("Done")

# Modelle Bewerten


In [ ]:
# Beste Modelle finden, r2_score ausgeben

# Nachdem sgd zu schlecht ist, wird es zur Evaluation entfernt!
if "sgd" in model["types"]:
    model["types"].remove("sgd")
    
for mtype in model["types"]:
    best = model["scores"][mtype].index(np.sort(model["scores"][mtype])[-1:][0])
    model["best"][mtype] = best    
    mean_score = np.mean(model["scores"][mtype])
    std_score = np.std(model["scores"][mtype])
    max_score = model["scores"][mtype][model["best"][mtype]]
    print("r2_score von %s:\t%.4f +- %.3f (%.4f max)" % (mtype, mean_score, std_score, max_score))


fig, big_axes = plt.subplots( figsize=(15, 10) , nrows=2, ncols=1)

big_axes[0].set_title("Verhältnis der vorhergesagten und wirklichen Preise",fontsize=16, pad=25)
big_axes[0].tick_params(labelcolor=(1.,1.,1., 0.0), top='off', bottom='off', left='off', right='off')
big_axes[0]._frameon = False

big_axes[1].set_title("Preisentwickung bei verschiedenen Hauszuständen",fontsize=16,pad=25)
big_axes[1].tick_params(labelcolor=(1.,1.,1., 0.0), top='off', bottom='off', left='off', right='off')
big_axes[1]._frameon = False



for i in range(len(model["types"])):
    mtype = model["types"][i]
    best_model = model["models"][mtype][model["best"][mtype]]
    x_test = model["data"][model["best"][mtype]]["x_test"]
    y_test = model["data"][model["best"][mtype]]["y_test"]
    pred_y = np.reshape(best_model.predict(poly.fit_transform(x_test)), (-1,1))
    
    # Abweichung in Prozent angeben
    ax = fig.add_subplot(2,len(model["types"]), i+1)
    plt.sca(ax)
    plt.hist(pred_y / y_test)
    plt.xlim(0.25,1.75)
    plt.ylim(0,75)
    plt.title(model["types"][i])
    plt.xlabel("Verhältnis der Werte")
    plt.ylabel("Anzahl")
    
    # Beispielvorhersage
    x = np.arange(100,200,1)
    y = []
    for c in range(5):
        y.append([])
        for v in x:
            y[c].append(best_model.predict(poly.fit_transform([[800 + 3*v, v, c , c, int(c/3), c, 2130, c, 2, 4]]))[0])
    
    ax = fig.add_subplot(2,len(model["types"]), i+1+len(model["types"]))
    plt.sca(ax)
    plt.xlim(100,200)
    plt.ylim(-100000,600000)
    plt.plot(x, y[0], c="yellow", label="1")     
    plt.plot(x, y[1], c="orange", label="2")
    plt.plot(x, y[2], c="red", label="3")
    plt.plot(x, y[3], c="purple" ,label="4")
    plt.plot(x, y[4], c="blue" ,label="5")
    plt.legend(loc="upper left", ncol=2)
    plt.title(model["types"][i])
    plt.xlabel("Wohnflaeche in Qm")
    plt.ylabel("Preis")
    
plt.tight_layout()
plt.show()

Wie in den Graphen zu sehen ist, ist das Verhältnis der vorhergesagten und wahren Preisen in einer Gaussverteilung, was zu erwarten war. Nachdem der Stochastig Gradiend Descend zu schlechte Werte vorhergesagt hat, wurde in der Analyse nicht weiter betrachtet.
In der zweiten Graphenreihe wurde ein fiktives Haus erstellt, dessen Zustand und Wohnfläche Variabel ist. In den Graphiken ist gut zusehen, dass es einen großen preislichen Unterschied zwischen den einzelnen Zuständen gibt, sehr gute Häuser sind deutlich teurer. In der Ordinarily Squared Loss- und Ridge-Regression werden Häuser mit sehr niedriger Qualität bei größer werdender Wohnfläche billiger, was entweder auf ein schlechtes Modell oder schlechte Testdaten hindeutet. Die Relation von Grundstücksfläche = 800 + 3 * Wohnfläche kann auch Einfluss auf die Qualität der Vorhersage nehmen. 

# Vergleich von Model und Testdaten

# Vorhersage 

In [ ]:
# "Grundstueck in qm", "Wohnflaeche in qm","ZustandN","KuecheN","Klimaanlage", "Gebaut", 
# "Garagenkapazitaet", "HeizungsqualitaetN", "Raeume"]

# Beste Regression wählen, für uns ist das das der Beste gradiend boosting Tree
best_model = model["models"]["gbt"][model["best"]["gbt"]]

y = best_model.predict(poly.fit_transform([[2000, 160, 4, 4, 1,4, 2130, 2, 4, 4]]))

print(round(y[0], 2), "GC Dollar")

# Unabhängige Validierung

In [ ]:
# housing_data2 = pd.read_csv('DatenAusgegeben1.2_UTF8_manuell.csv', sep=";", keep_default_na=False)


# housing_data2["Preisproqm"] = housing_data2["Preis"]/housing_data2["Wohnflaeche in qm"]

# # Monate seit 2136
# housing_data2["Monate"] = (housing_data2["Verkaufsjahr"]-2136)*12 + housing_data2["Verkaufsmonat"] -1

# # Werte der Heizungsqualitaet in Zahlen umwandeln
# housing_data2["Heizungsqualitaet"] = [1 if x=="Schl" else 2 if x=="Ud" else 3 if x=="Ty" else 4 if x=="Gut" else 5 for x in housing_data2["Heizungsqualitaet"]]

# # Zustand, 1-10 wird auf 1-5 gemapt, das ist zwar ein bisschen ungenauer, aber einfacher zu vergleichen
# housing_data2["Zustandf"] = [int(x/2) + x%2 for x in housing_data2["Zustand"]]

# # Alle Zustände werden von 1...X auf 0....X-1 geschoben
# housing_data2["HeizungsqualitaetN"] = [ x-1 for x in housing_data2["Heizungsqualitaet"]]
# housing_data2["ZustandN"] = [ x-1 for x in housing_data2["Zustandf"]]
# housing_data2["FassadeN"] = [ x-1 for x in housing_data2["Zustand Fassade"]]
# housing_data2["KuecheN"] = [ x-1 for x in housing_data2["Kuechenqualitaet"]]

# # Klimaanlage Ja/Nein  -> 1/0
# housing_data2["Klimaanlage"] = [1 if x == "Y" else 0 for x in housing_data2["Klimaanlage"]]

# # Besonders Große Daten beim Preis + Wohnfläche in qm rausschmeißen
# max_value = housing_data2["Preis"].unique()[-1:][0]
# housing_data2[housing_data2["Preis"] != max_value]

# max_value = housing_data2["Wohnflaeche in qm"].unique()[-1:][0]
# housing_data2[housing_data2["Wohnflaeche in qm"] != max_value]

# max_value = housing_data2["Grundstueck in qm"].unique()[-2:][0]
# housing_data2[housing_data2["Grundstueck in qm"] != max_value]

# # NA in Garage Typ wird zu "Keine Garage"
# housing_data2["Garage Typ"] = ['keine Garage' if x=='NA' else x for x in housing_data2["Garage Typ"]]


# X2 = {}
# for feature in features:
#     X2[feature] = housing_data2[feature]
# X2 = pd.DataFrame(X)

# Y2 = pd.DataFrame(housing_data2[predict])

# x2_poly = poly.fit_transform(X2.iloc[train, :])
# poly.fit(X2.iloc[train, :], Y2.iloc[train])
# x2_poly_test = poly.fit_transform(X2.iloc[test, :])

# # R2, MSE, RMSE, MAPE, MAX
# test_model = model["models"]["gbt"][model["best"]["gbt"]]
# x_test = model["data"][model["best"]["dasd"]]["x_poly_test"]
# y_test = model["data"][model["best"]["gbt"]]["y_test"]

# r2_test = r2_score(test_model.predict(x_test), y_test)
# mse_test = mean_squared_error(test_model.predict(x_test), y_test)
# rmse_test = mean_squared_error(test_model.predict(x_test), y_test, squared=False)
# mape_test = mape(test_model.predict(x_test), y_test)
# max_test = max_error(test_model.predict(x_test), y_test)

# print("R2 Wert:", r2_test)
# print("Mean Squared Error Wert:", mse_test)
# print("Negativer Mean Square Error Wert:", rmse_test)
# print("Mean Absolute Percentage Error Wert:", mape_test)
# print("Max Error Wert:", max_test)

# Evaluation GBT


Wie bewerten wir den GBT? Metriken https://docs.aws.amazon.com/de_de/machine-learning/latest/dg/evaluating_models.html

R2 Wert: 0.8860877506932229 0.8397 +- 0.025 (0.8861 max)
Mean Squared Error Wert: 553102813.4308888
Negativer Mean Square Error Wert: 23518.13796691585
Mean Absolute Percentage Error Wert: 44.416836783323326
Max Error Wert: 121233.35705050646

Summary (3 - 5 Sätze)

# Veraltete Regression

In [ ]:
while 0 != 0:
    # Daten vorbereiten

    # Wichtigste Merkmale 
    # Klimaanlage, Heizungsqualität, Küchenqualität, Zustand, Fassadenqualität 

    # In Trainings- und Testdaten unterteilen
    X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2)

    poly = PolynomialFeatures(degree=3)
    x_poly = poly.fit_transform(X_train)
    poly.fit(X_train, y_train)


    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
    # Regression

    reg = linear_model.LinearRegression()
    ridge = linear_model.Ridge(alpha=0.05)
    lasso = linear_model.Lasso(alpha=0.05)

    reg.fit(x_poly, y_train)
    ridge.fit(x_poly, y_train)
    lasso.fit(x_poly, y_train)

    #V alidierung
    y_pred_reg = reg.predict(poly.fit_transform(X_test))
    y_pred_ridge = ridge.predict(poly.fit_transform(X_test))
    y_pred_lasso = lasso.predict(poly.fit_transform(X_test))


    print(r2_score(y_test, y_pred_reg), mean_squared_error(y_test, y_pred_reg), max_error(y_test, y_pred_reg))
    print(r2_score(y_test, y_pred_ridge), mean_squared_error(y_test, y_pred_ridge), max_error(y_test, y_pred_ridge))
    print(r2_score(y_test, y_pred_lasso), mean_squared_error(y_test, y_pred_lasso), max_error(y_test, y_pred_lasso))

# Anleitung

In [ ]:
def mpredict(grundstück, wohnraum, zustand, küche, heizung, fassade, klimaanlage, gebaut, garagenkapazität, räume):
    mtyp = "gbt"
    dinput = ["wohnraum", "grundstück", "zustand", "küche", "heizung", "fassade", "klimaanlage", "gebaut", "garagenkapazität", "räume"]
    data = []
    curr_model = model["models"][mtyp][model["best"][mtyp]]
    # Eingaben überprüfen
    if wohnraum < 100 or wohnraum > 400:
        print("Der Wohnraum liegt außerhalb des Trainingsbereichs, Vorhersagen können fehlerhaft sein")
    data.append(wohnraum)
    if grundstück < 800 or grundstück > 5000: 
        print("Die Grundstücksfläche liegt außerhalb des Trainingsbereichs, Vorhersagen können fehlerhaft sein")
    data.append(grundstück)
    if zustand < 0 or zustand > 5:
        print("Der Zustand liegt außerhalb des Wertebereichs, es kann keine Vorhersage getroffen werden")
    data.append(zustand)
    data.append(küche)
    data.append(klimaanlage)
    data.append(fassade)
    data.append(gebaut)
    data.append(garagenkapazität)
    data.append(heizung)
    data.append(räume)
    pred_y = curr_model.predict(poly.fit_transform([data]))[0]
    print("Vorhergesagter Preis: %i€" % int(pred_y))
    #"Grundstueck in qm", "Wohnflaeche in qm","ZustandN","KuecheN","Klimaanlage", "Fassade", "Gebaut", 
    #"Garagenkapazitaet", "HeizungsqualitaetN", "Raeume"]

Anhand der vorhandenen Daten wurde ein Modell entwickelt, mit dem der preisliche Mehraufwand von Renovationen mit den größeren Verkaufskosten verglichen werden kann. Für diese Aufgaben wurden Modelle wurde auf über 2000 Datensätzen trainiert und getestet, und die Besten zu einer manuellen Untersuchung herausgesucht. Das beste Modell war ein Gradiend Boosting Tree mit einem r2-Score von 0.88 #TODO, ist das noch richtig?
Das Modell bietet herausragende Vorhersagemöglichkeiten für Häuser im Preisbereich von 100.000€ bis 400.000€ 
#sollen wir hier mehrere Grenzen setzen?? -> Grenzen finden?
Sollten Werte getestet werden, die außerhalb der definierten Randgebiete liegen, werden zwar immernoch gute Preisvorschläge generiert, ihre Richtigkeit kann aber nicht mehr garantiert werden.

Um eine Vorhersage generiert zu bekommen, werden folgende Daten benötigt:
- Grundstücksfläche in Qm
- Wohnfläche in Qm
- Zustand des Hauses (Zahl von 1 - 5)
- Zustand der Küche (Zahl von 1 - 5)
- Vorhandensein einer Klimaanlage (0: Nein, 1: Ja)
- Baujahr
- Garagenkapazität
- Zustand der Heizung (Zahl von 1 - 5)
- Zustand der Fassade (Zahl von 1 - 5)
- Anzahl der Räume

ein Beispielaufruf kann so aussehen:
mpredict(3000, 180, 4, 4, 1, 4, 2130, 2, 2, 20)


In [ ]:
#"Grundstueck in qm", "Wohnflaeche in qm","ZustandN","KuecheN","Klimaanlage", "Gebaut", #"Garagenkapazitaet", "HeizungsqualitaetN", "Raeume"]

mpredict(1000,120,4,4,1,4,2103,2,4,4)

In [ ]:
#TODO: Alle Modelle gbt nehmen und mittelwert bilden???